In [3]:
!pip install trtokenizer
!pip install TurkishStemmer

In [4]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from snowballstemmer import stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
from TurkishStemmer import TurkishStemmer



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
df = pd.read_excel('tweetData.xlsx')

In [6]:
df = df.dropna()

In [7]:
df.shape

(1976, 2)

In [8]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [9]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

In [10]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [11]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [12]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []

# Stop kelimeleri temizleme işlevi
def remove_stopwords_nltk(words):
    stop_words = set(stopwords.words('turkish'))
    return [word for word in words if word.lower() not in stop_words]

def turkishstemmer(text):
  stemmer = TurkishStemmer()
  # Stem işlemi
  stemmed_words = [stemmer.stem(word) for word in text]
  return ' '.join(stemmed_words)




# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_nltk)
df['lemmatized_words'] = df['cleaned_words'].apply(turkishstemmer)

# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [13]:
# Kelimeleri tokenize et
tokenize_metni = [metin.lower().split() for metin in df['Tweets']]

# Word2Vec modelini eğit
word2vec_model = Word2Vec(sentences=tokenize_metni, vector_size=100, window=5, min_count=1, workers=4)

# Her metin için ortalama kelime vektörlerini hesapla
def ortalama_vektor(metin):
    kelimeler = metin.lower().split()
    vektörler = [word2vec_model.wv[kelime] for kelime in kelimeler if kelime in word2vec_model.wv]
    return sum(vektörler) / len(vektörler) if vektörler else [0]*100

df['word2vec_vektör'] = df['Tweets'].apply(ortalama_vektor)

print(df['word2vec_vektör'])

0        [-0.00092426344, 0.0026438497, 0.0027702502, -...
1        [-0.0041658175, 0.0076010004, -0.00034480318, ...
2        [-0.0012956215, 0.0051167565, -0.00089885964, ...
3        [-0.004659762, 0.007955334, 0.0010650909, 0.00...
4        [-0.0045872577, 0.0057900995, 0.004198971, 0.0...
5        [-0.0013090089, 0.0032775209, -0.0005851213, -...
6        [-0.0038758784, 0.0098550655, -0.00011908866, ...
7        [-0.00063532544, 0.0054671774, 0.0012116748, -...
8        [-0.0025945217, 0.01114676, -0.0018600647, 0.0...
9        [-0.0026213261, 0.0071163177, -0.0003957667, 0...
10       [-0.0029991418, 0.006673733, -0.0003764623, 0....
11       [-0.0048383675, 0.0050708344, 0.0012740622, 0....
12       [-0.0056877937, 0.009341928, -0.0002373039, 0....
13       [-0.007051296, 0.0059669088, 0.002565967, 0.00...
14       [-0.0016141182, 0.0059531494, 0.001153131, -0....
15       [-0.0030246242, 0.004981266, 0.0011075223, 0.0...
16       [-0.003176544, 0.007951118, -0.00053076464, 0..